<a href="https://colab.research.google.com/github/ayandalab/The_Kvasir_Capsule_Dataset/blob/main/The_Kvasir_Capsule_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1yDcNMTI-8Zq2Mvs8qe7pLNpPx4BW5Yta',
dest_path='content/The Kvasir-Capsule.zip',
unzip=True)

Unzipping...Done.


In [2]:
import pandas as pd # Data analysis and manipultion tool
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf # Deep Learning Tool
import os # OS module in Python provides a way of using operating system dependent functionality
import cv2 # Library for image processing
from sklearn.model_selection import train_test_split # For splitting the data into train and validation set
from sklearn.metrics import f1_score

In [3]:
labels = pd.read_csv("/content/content/The Kvasir-Capsule/Training_set.csv") # loading the labels
labels.head() # will display the first five rows in labels dataframe

,filename,label
0,Image_1.jpg,Normal clean mucosa
1,Image_2.jpg,Normal clean mucosa
2,Image_3.jpg,Foreign body
3,Image_4.jpg,Normal clean mucosa
4,Image_5.jpg,Normal clean mucosa


In [4]:
labels.tail() # will display the last five rows in labels dataframe

,filename,label
37785,Image_37786.jpg,Normal clean mucosa
37786,Image_37787.jpg,Normal clean mucosa
37787,Image_37788.jpg,Normal clean mucosa
37788,Image_37789.jpg,Ileocecal valve
37789,Image_37790.jpg,Ileocecal valve


In [5]:
file_paths = [[fname, '/content/content/The Kvasir-Capsule/train/' + fname] for fname in labels['filename']]

In [6]:
# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
  print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
  print('Number of labels does not match the number of filenames')

Number of labels i.e.  37790 matches the number of filenames i.e.  37790


In [7]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,/content/content/The Kvasir-Capsule/train/Imag...
1,Image_2.jpg,/content/content/The Kvasir-Capsule/train/Imag...
2,Image_3.jpg,/content/content/The Kvasir-Capsule/train/Imag...
3,Image_4.jpg,/content/content/The Kvasir-Capsule/train/Imag...
4,Image_5.jpg,/content/content/The Kvasir-Capsule/train/Imag...


In [8]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()

,filename,filepaths,label
0,Image_1.jpg,/content/content/The Kvasir-Capsule/train/Imag...,Normal clean mucosa
1,Image_2.jpg,/content/content/The Kvasir-Capsule/train/Imag...,Normal clean mucosa
2,Image_3.jpg,/content/content/The Kvasir-Capsule/train/Imag...,Foreign body
3,Image_4.jpg,/content/content/The Kvasir-Capsule/train/Imag...,Normal clean mucosa
4,Image_5.jpg,/content/content/The Kvasir-Capsule/train/Imag...,Normal clean mucosa


In [9]:
def f(r):
  if r == "ampulla_of_vater":
    return 0
  elif r == "Angiectasia":
    return 1
  elif r == "Blood - fresh":
    return 2
  elif r == "Blood - hematin":
    return 3
  elif r == "Erosion":
    return 4
  elif r == "Erythema":
    return 5
  elif r == "Foreign body":
    return 6
  elif r == "Ileocecal valve":
    return 7
  elif r == "Lymphangiectasia":
    return 8
  elif r == "Normal clean mucosa":
    return 9
  elif r == "Polyp":
    return 10
  elif r == "Pylorus":
    return 11
  elif r == "Reduced mucosal view":
    return 12
  else:
    return 13

train_data['label'] = train_data['label'].apply(f)

In [10]:
data = [] # initialize an empty numpy array
image_size = 100 # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):

  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
  data.append([new_img_array, train_data['label'][i]])

In [11]:
# image pixels of a image
data[0]

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 9]

In [12]:
np.random.shuffle(data)

In [13]:
x = []
y = []
for image in data:
  x.append(image[0])
  y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y) 

In [14]:
np.unique(y, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
 array([    8,   693,   357,    10,   405,   127,   621,  3351,   473,
        27470,    44,  1223,  2325,   683]))

In [15]:
x = x.reshape(-1, 100, 100, 1)

In [16]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

In [17]:
cnn = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)),
tf.keras.layers.MaxPooling2D((2, 2)),

tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
tf.keras.layers.MaxPooling2D((2, 2)),

# tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(64, activation='relu'),
tf.keras.layers.Dense(14, activation='sigmoid')
])

In [18]:
cnn.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])

In [19]:
cnn.fit(X_train, y_train, epochs=10, batch_size=20)

Epoch 1/10
1323/1323 [==============================] - 312s 235ms/step - loss: 1.7010 - accuracy: 0.7240
Epoch 2/10
1323/1323 [==============================] - 309s 234ms/step - loss: 1.0404 - accuracy: 0.7286
Epoch 3/10
1323/1323 [==============================] - 307s 232ms/step - loss: 0.9421 - accuracy: 0.7463
Epoch 4/10
1323/1323 [==============================] - 303s 229ms/step - loss: 0.8164 - accuracy: 0.7746
Epoch 5/10
1323/1323 [==============================] - 301s 228ms/step - loss: 0.6666 - accuracy: 0.8133
Epoch 6/10
1323/1323 [==============================] - 300s 227ms/step - loss: 0.5450 - accuracy: 0.8447
Epoch 7/10
1323/1323 [==============================] - 302s 228ms/step - loss: 0.4490 - accuracy: 0.8707
Epoch 8/10
1323/1323 [==============================] - 300s 227ms/step - loss: 0.3549 - accuracy: 0.8989
Epoch 9/10
1323/1323 [==============================] - 298s 225ms/step - loss: 0.2956 - accuracy: 0.9170
Epoch 10/10
1323/1323 [=======================

In [20]:
cnn.evaluate(X_val, y_val)

355/355 [==============================] - 32s 89ms/step - loss: 1.8100 - accuracy: 0.8035


[1.8100063800811768, 0.8034753203392029]

In [21]:
test_image_order = pd.read_csv("/content/content/The Kvasir-Capsule/Testing_set.csv")
file_paths = [[fname, '/content/content/The Kvasir-Capsule/test/' + fname] for fname in test_image_order['filename']]
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])

In [22]:
test_pixel_data = [] # initialize an empty numpy array
image_size = 100 # image size taken is 100 here. one can take other size too
for i in range(len(test_images)):

  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array

  test_pixel_data.append(new_img_array)

In [23]:
test_pixel_data = np.array(test_pixel_data)
test_pixel_data = test_pixel_data.reshape(-1, 100, 100, 1)
pred = cnn.predict(test_pixel_data)

In [24]:
prediction = []
for value in pred:
  prediction.append(np.argmax(value))

predictions = []
for r in prediction:
  if r == 0:
    predictions.append("ampulla_of_vater")
  elif r == 1:
    predictions.append("Angiectasia")
  elif r == 2:
    predictions.append("Blood - fresh")
  elif r == 3:
    predictions.append("Blood - hematin")
  elif r == 4:
    predictions.append("Erosion")
  elif r == 5:
    predictions.append("Erythema")
  elif r == 6:
    predictions.append("Foreign body")
  elif r == 7:
    predictions.append("Ileocecal valve")
  elif r == 8:
    predictions.append("Lymphangiectasia")
  elif r == 9:
    predictions.append("Normal clean mucosa")
  elif r == 10:
    predictions.append("Polyp")
  elif r == 11:
    predictions.append("Pylorus")
  elif r == 12:
    predictions.append("Reduced mucosal view")
  else:
    predictions.append("ulcer")

In [25]:
res = pd.DataFrame({'label': predictions}) # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)

# To download the csv file locally
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>